In [1]:
import pandas as pd
import numpy as np
import re
import sys, getopt, os
import csv
import pickle
import copy

pd.set_option('display.max_rows', 500)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline  
import seaborn as sns
sns.set_style("darkgrid")

%run SodaKick_download_functions.ipynb

In [2]:
#note xgboost can handle missing values internally
#list of parameters:
#https://xgboost.readthedocs.io/en/latest/parameter.html
#https://shengyg.github.io/repository/machine%20learning/2017/02/25/Complete-Guide-to-Parameter-Tuning-xgboost.html

#custom obj functions
#https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html
#https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters

In [3]:
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

from sklearn.model_selection import RepeatedKFold, KFold
from sklearn.multioutput import MultiOutputRegressor

import xgboost as xgb
from xgboost import XGBRegressor

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.xgboost import TuneReportCheckpointCallback
from functools import partial 
from ray.tune.suggest.hyperopt import HyperOptSearch

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
#https://docs.ray.io/en/master/tune/tutorials/tune-xgboost.html

from hyperopt import hp

In [4]:
#https://towardsdatascience.com/quirky-keras-custom-and-asymmetric-loss-functions-for-keras-in-r-a8b5271171fe
def WSE(output, target, a=1.5, b=.5):
    loss = torch.mean(a*torch.minimum(torch.zeros(output.shape[1]),output - target)**2+\
                      b*torch.maximum(torch.zeros(output.shape[1]),output - target)**2)      
    return loss

def WSEl1(output, target, a=1.5, b=.5):
    loss = torch.mean(a*torch.abs(torch.minimum(torch.zeros(output.shape[1]),output - target))+\
                      b*torch.abs(torch.maximum(torch.zeros(output.shape[1]),output - target)))      
    return loss

def WSE2(output, target, a=1.5, b=.5):
    loss = np.mean(a*np.minimum(np.zeros(output.shape[0]),output - target)**2+\
                      b*np.maximum(np.zeros(output.shape[0]),output - target)**2)      
    return loss

def WSEl12(output, target, a=1.5, b=.5):
    loss = np.mean(a*np.abs(np.minimum(np.zeros(output.shape[0]),output - target))+\
                      b*np.abs(np.maximum(np.zeros(output.shape[0]),output - target)))      
    return loss

In [18]:
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_out_2a.pkl', 'rb') as pk:
    out=pickle.load(pk) 
    
out[:,200-16:200-8].sum()

12.0

In [134]:
from sklearn import preprocessing

    
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_inp_2a.pkl', 'rb') as pk:
    inp=pickle.load(pk)
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_out_2a.pkl', 'rb') as pk:
    out=pickle.load(pk)     
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_inp_2b.pkl', 'rb') as pk:
    inpb=pickle.load(pk)
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_out_2b.pkl', 'rb') as pk:
    outb=pickle.load(pk)     

inp=np.concatenate([inp,inpb])
out=np.concatenate([out,outb])

def normalize_mins(vec):
    for i in range(vec.shape[0]):
        vec[i][::8]=vec[i][::8]/90

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def NormalizeMatrix(data):   
    for i in range(data.shape[1]):
        data[:,i] = NormalizeData(data[:,i])
        
NormalizeMatrix(inp)
np.nan_to_num(inp, copy=False)

normalize_mins(out)

#scaler = preprocessing.StandardScaler().fit(inp)
#inp = scaler.transform(inp)

//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


In [135]:
from typing import Tuple

def WSE(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
    
    target = dtrain.get_label()
    predt[predt < -1] = -1 + 1e-6
    
    a=1.5
    b=.5
    
    elements = a*np.minimum(np.zeros(len(predt)),predt - target)**2+\
                      b*np.maximum(np.zeros(len(predt)),predt - target)**2
    
    return 'WSE', float(np.sqrt(np.sum(elements) / len(target)))

In [136]:
search_space = {
 "n_estimators": 50,
 "max_depth": hp.choice("max_depth",np.linspace(8,15,8, dtype=int)),
 #"max_depth": hp.choice("max_depth",np.linspace(1,15,8, dtype=int)),
 "min_child_weight": hp.choice("min_child_weight",[1, 2, 3, 4, 5]),
 "subsample": hp.choice("subsample",np.linspace(.6,.9,4)),
 #"subsample": hp.choice("subsample",np.linspace(.5,.9,5)),
 "eta": hp.choice("eta",[1e-2, 5e-2, 1e-1, 5e-1, 3e-1]),
 "colsample_bytree": hp.choice("colsample_bytree",np.linspace(0.1,.9,5)),
 "alpha": hp.randint("alpha", 3),
 #"alpha": hp.randint("alpha", 5),
 "lambda": hp.choice("lambda", np.linspace(0,10,6)),
 #"gamma": hp.choice("gamma",np.linspace(0,.9,4)),
 "objective": "reg:pseudohubererror",
 "eval_metric": "rmse", 
 "learning_rate": 1e-1, 
 }

x_train, x_test, y_train, y_test = train_test_split(
     inp, out, test_size=0.3, shuffle=True)

x_train, x_test, y_train, y_test = train_test_split(
     x_train, y_train, test_size=0.3, shuffle=True)

def train_xgb(config, col=0):
    
    evals=[]
    models=[]
    for k in range(50):
        models.append(XGBRegressor(**config))
        eval_set = [(x_train, y_train[:,col+8*k]),(x_test, y_test[:,col+8*k])]
        models[-1].fit(x_train, y_train[:,col+8*k], eval_metric="rmse", eval_set=eval_set, early_stopping_rounds = 3, verbose=False)    
        evals_result = models[-1].evals_result()   
        evals.append(evals_result['validation_1']['rmse'][-1]**2)
    
    #preds = model.predict(x_test,ntree_limit=model.get_booster().best_ntree_limit)    
    tune.report(rmse = np.sqrt(np.mean(evals)), done=True)    

def tune_xgb(search_space, col):
    
    search_alg = HyperOptSearch(space=search_space, metric="rmse", mode="min")

    # This will enable aggressive early stopping of bad trials.
    #scheduler = ASHAScheduler(
    #    max_t=20,  # 10 training iterations
    #    grace_period=5,
    #    reduction_factor=2)

    analysis = tune.run(
        partial(train_xgb, col=col),
        search_alg=search_alg,
        resources_per_trial={"cpu": 1},
        num_samples=25,
        #scheduler=scheduler,
        verbose=0,
        metric="rmse", mode="min")

    return analysis

In [23]:
#best_bst=[]
best_cfg=[]
for i in range(8):
    print('Feature #: '+str(i))
    analysis = tune_xgb(search_space, i)
    print(analysis)
    best_cfg.append(analysis.get_best_config())
    #best_cfg[-1]['file']=analysis.best_checkpoint
    #best_bst.append(get_best_model_checkpoint(analysis))
    print('\n')

2021-06-02 12:46:22,664	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-06-02 12:46:22,665	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


Feature #: 0


2021-06-02 12:46:23,803	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 59262864 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
(pid=8646) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=8646)   "because it will generate extra copies and increase " +
2021-06-02 12:46:27,051	WARNING util.py:162 -- The `start_trial` operation took 0.567 s, which may be a performance bottleneck.
(pid=8834) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=8834)   "because it will generate extra copies and increase " +
2021-06-02 12:46:28,170	WARNING util.py:162 -- The `start_tri



Feature #: 1


2021-06-02 14:23:46,176	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 59262864 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-06-02 14:23:46,246	WARNING util.py:162 -- The `start_trial` operation took 0.502 s, which may be a performance bottleneck.
(pid=9963) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=9963)   "because it will generate extra copies and increase " +
2021-06-02 14:23:49,132	WARNING util.py:162 -- The `start_trial` operation took 0.595 s, which may be a performance bottleneck.
(pid=9967) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(p

...

2021-06-02 19:11:03,916	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 59262864 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-06-02 19:11:03,995	WARNING util.py:162 -- The `start_trial` operation took 0.503 s, which may be a performance bottleneck.
(pid=13765) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=13765)   "because it will generate extra copies and increase " +
(pid=13773) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=13773)   "because it will generate extra copies and increase " +
2021-06-02 19:11:08,097	WARNING util.py:162 -- The `start



Feature #: 6


2021-06-02 19:44:39,267	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 59262864 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
(pid=14125) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=14125)   "because it will generate extra copies and increase " +
(pid=14128) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=14128)   "because it will generate extra copies and increase " +
(pid=14131) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies an



Feature #: 7


2021-06-02 20:07:14,188	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 59262864 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
(pid=14509) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=14509)   "because it will generate extra copies and increase " +
(pid=14516) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=14516)   "because it will generate extra copies and increase " +
2021-06-02 20:07:18,185	WARNING util.py:162 -- The `start_trial` operation took 0.558 s, which may be a performance bottleneck.
(pid=14519) //miniconda3/lib/python3.7/site-packages/xgbo

In [11]:
stats=pd.concat([pd.Series(bc) for bc in best_cfg], axis=1).T
stats

,alpha,colsample_bytree,eta,eval_metric,gamma,lambda,learning_rate,max_depth,min_child_weight,n_estimators,objective,subsample
0,1,0.3,0.3,rmse,0.0,0.0,0.1,13,1,203,reg:pseudohubererror,0.9
1,2,0.5,0.5,rmse,0.0,10.0,0.1,13,2,107,reg:pseudohubererror,0.8
2,1,0.9,0.05,rmse,0.0,0.0,0.1,13,2,216,reg:pseudohubererror,0.8
3,0,0.7,0.3,rmse,0.0,8.0,0.1,9,3,452,reg:pseudohubererror,0.9
4,0,0.9,0.05,rmse,0.0,6.0,0.1,11,5,386,reg:pseudohubererror,0.8
5,2,0.9,0.05,rmse,0.0,0.0,0.1,9,4,211,reg:pseudohubererror,0.7
6,3,0.3,0.05,rmse,0.0,4.0,0.1,13,4,425,reg:pseudohubererror,0.9
7,1,0.1,0.5,rmse,0.0,6.0,0.1,13,1,419,reg:pseudohubererror,0.8


In [43]:
#with open(r'/Users/federico comitani/GitHub/sodakick/data/best_cfg.pkl', 'wb') as file:
#    pickle.dump(best_cfg,file)

In [137]:
with open(r'/Users/federico comitani/GitHub/sodakick/data/best_cfg.pkl', 'rb') as file:
    best_cfg=pickle.load(file)

In [138]:
"""for i in range(len(best_cfg)):
    best_cfg[i]['eta']=.3
    best_cfg[i]['subsample']=.8  
    best_cfg[i]['alpha']=10
    best_cfg[i]['max_depth']=7
    best_cfg[i]['colsample_bytree']=.3"""

"for i in range(len(best_cfg)):\n    best_cfg[i]['eta']=.3\n    best_cfg[i]['subsample']=.8  \n    best_cfg[i]['alpha']=10\n    best_cfg[i]['max_depth']=7\n    best_cfg[i]['colsample_bytree']=.3"

# one by one

In [143]:
x_train, x_test, y_train, y_test = train_test_split(inp, out, test_size=0.20, random_state=32)

models=[]
results=[]
epcs=[]
losses=[]
for num in range(50):
    print()
    print(num)
    print('---------')
    for num2 in range(8):
        print(num2, end='\t')
        
        tmpparms = best_cfg[num2]
        tmpparms.pop("file", None)    
        tmpparms['n_estimators']=1000
        #print(tmpparms)
        
        models.append(XGBRegressor(**tmpparms))
        models[-1].fit(x_train, y_train[:,num*8+num2],
            eval_set = [(x_train, y_train[:,num*8+num2]),(x_test, y_test[:,num*8+num2])],
            eval_metric = 'rmse',
            early_stopping_rounds = 10, verbose=False)
        
        results.append(models[-1].evals_result())
        best_iteration = models[-1].get_booster().best_ntree_limit
        
        epcs.append(models[-1].best_iteration)
        losses.append(models[-1].get_booster().best_score)
        
        print(epcs[-1],end='\t')
        print(losses[-1])
        
        
#with open(r'/Users/federico comitani/GitHub/sodakick/data/models.pkl', 'wb') as file:
#    pickle.dump(models,file)


0
---------
0	192	0.038187
1	87	0.024606
2	192	0.014929
3	372	0.091017
4	881	0.005275
5	235	0.043508
6	524	0.455267
7	474	0.764189

1
---------
0	297	0.076361
1	380	0.106076
2	359	0.085836
3	314	0.174082
4	647	0.054326
5	371	0.05213
6	62	0.024795
7	151	0.010393

2
---------
0	275	0.060912
1	484	0.100173
2	383	0.113056
3	359	0.187725
4	497	0.04107
5	336	0.024715
6	142	0.000609
7	128	0.000229

3
---------
0	408	0.062566
1	409	0.195817
2	307	0.188426
3	370	0.179297
4	742	0.041453
5	249	0.024965
6	142	0.000609
7	128	0.000229

4
---------
0	281	0.06991
1	400	0.206256
2	354	0.153006
3	352	0.157666
4	558	0.039223
5	212	0.025755
6	142	0.000609
7	128	0.000229

5
---------
0	335	0.074312
1	406	0.142714
2	327	0.159187
3	325	0.172329
4	686	0.012786
5	263	0.024661
6	142	0.000609
7	128	0.000229

6
---------
0	424	0.079573
1	432	0.171535
2	394	0.121434
3	318	0.165287
4	758	0.033697
5	248	0.01949
6	142	0.000609
7	128	0.000229

7
---------
0	363	0.054471
1	335	0.147922
2	308	0.166155
3	313	0.20847
4	4

In [155]:
def multiprediction(inpt, models):
    
    preds=[]
    for model in models:
        preds.append(model.predict(inpt))
        
    return np.array(preds).T   

In [156]:
y_pred = multiprediction(x_test, models)

from sklearn.metrics import mean_squared_error as mse
mse(y_pred,y_test)

0.010354236220101428

In [157]:
print('Baseline WSE: {:.3f}'.format(WSE2(np.array([0]*y_train[0].shape[0]),y_train[0])))
print('Baseline WSE l1: {:.3f}'.format(WSEl12(np.array([0]*y_train[0].shape[0]),y_train[0])))
print('Baseline MSE: {:.3f}'.format(WSE2(np.array([0]*y_train[0].shape[0]),y_train[0], a=1, b=1)))
print('Baseline MSE l1: {:.3f}'.format(WSEl12(np.array([0]*y_train[0].shape[0]),y_train[0], a=1, b=1)))

print((y_train[1]-y_train[10]).sum())
print((y_train[50]-y_train[60]).sum())
print((y_train[100]-y_train[110]).sum())

print('Baseline WSE: {:.3f}'.format(WSE2(y_pred[0],y_train[0])))
print('Baseline WSE l1: {:.3f}'.format(WSEl12(y_pred[0],y_train[0])))
print('Baseline MSE: {:.3f}'.format(WSE2(y_pred[0],y_train[0], a=1, b=1)))
print('Baseline MSE l1: {:.3f}'.format(WSEl12(y_pred[0],y_train[0], a=1, b=1)))

print((y_pred[1]-y_pred[10]).sum())
print((y_pred[50]-y_pred[60]).sum())
print((y_pred[100]-y_pred[110]).sum())

Baseline WSE: 0.255
Baseline WSE l1: 0.158
Baseline MSE: 0.170
Baseline MSE l1: 0.105
2.5444444444444443
-1.0
-9.0
Baseline WSE: 0.123
Baseline WSE l1: 0.079
Baseline MSE: 0.101
Baseline MSE l1: 0.074
-1.1469386
-5.979491
5.7390623


In [ ]:
def revert_output(output,lineup=None):

    
    reframe=pd.DataFrame(copy.deepcopy(output.reshape(50,8)),
                 columns=['minutes','goals','assists','cards_yellow','cards_red','own_goals','goals_against','saves'])
    
    reframe[reframe<0] = 0
    if lineup is not None:
        reframe.index=lineup
        reframe.drop([x for x in reframe.index if x.startswith('dummy')], axis=0, inplace=True)
        
    reframe['minutes']*=90
    #byteamframe=pd.concat([reframe.loc[[x for x in reframe.index if x in lineup[lineup['team']==0].index]].sum(axis=0),
    #                    reframe.loc[[x for x in reframe.index if x in lineup[lineup['team']==1].index]].sum(axis=0)], axis=1).T
    
    byteamframe=pd.concat([reframe.iloc[:25,:].sum(axis=0),reframe.iloc[25:,:].sum(axis=0)], axis=1).T
    
    return reframe, byteamframe[byteamframe.columns[1:]]

In [162]:
cats=['minutes','goals','assists','cards_yellow','cards_red','own_goals']+['goals_against','saves']

reframe, byteamframe = revert_output(y_pred[4])
print(byteamframe)
reframe, byteamframe = revert_output(y_test[4])
print(byteamframe)

      goals   assists  cards_yellow  cards_red  own_goals  goals_against  \
0  1.832174  1.568414      2.981649   0.361313   0.258550       1.329285   
1  1.874772  1.659282      3.040978   0.185841   0.052375       0.891357   

      saves  
0  4.218348  
1  3.395755  
   goals  assists  cards_yellow  cards_red  own_goals  goals_against  saves
0    2.0      2.0           2.0        0.0        0.0            2.0    4.0
1    2.0      0.0           2.0        0.0        0.0            2.0    1.0
